In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
import ConnectionConfig as cc
cc.setupEnvironment()

### Config stuff

# Creating the operational database
In order to run this demo you have to create a tutorial_op database and run the PostgreSQL_SalesOperational.sql script.

### Connection properties
ConnectionConfig.py (cc) is created and imported to simplify the database connection process.
Consult the file to get more insights.

### Session setup
"spark.driver.extraClassPath" is added. This is needed to include the necessary jars when running the sparkJobs.

In [3]:
from delta import configure_spark_with_delta_pip

builder = SparkSession.builder \
    .appName("DBConnectionTest") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .master("local[4]")
#This one must be added to be able to query a jdbc source
#extra_packages = ["org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2","com.microsoft.sqlserver:mssql-jdbc:12.2.0.jre8"]
extra_packages = ["org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2","org.postgresql:postgresql:42.7.4"]

builder = configure_spark_with_delta_pip(builder, extra_packages=extra_packages)

spark = builder.getOrCreate()
builder.getOrCreate()
spark.sparkContext.setLogLevel("DEBUG")

### Reading a JDBC table
Read a table from sqlServer connection

#### Using the ConnectionConfig (cc) to make things easy
cc can make a connection url based on a connection profile in config.ini. To do this, first set the name of the connection.

#### Partitioning
As Spark is build to work in parallel reading from the database can also be done in parallel. In this case we define 4 partitions. Spark has to know how to split the data for every partition. Therefore you have to provide a partition column and a lower and upperbound. In this case the  on of the 4 queries that Spark will fire looks like "select * from dbo.sales where Order_ID <= 500 and Order_id > 250"

In [6]:
cc.set_connectionProfile("tutorial_op")
print(cc.create_jdbc())
sales_df = spark.read \
    .format("jdbc") \
    .option("driver" , "org.postgresql.Driver") \
    .option("url", cc.create_jdbc()) \
    .option("dbtable", "sales") \
    .option("user", cc.get_Property("username")) \
    .option("password", cc.get_Property("password")) \
    .option("partitionColumn", "Order_ID") \
    .option("numPartitions", 4) \
    .option("lowerBound", 0) \
    .option("upperBound", 1001) \
    .load()
sales_df.show(1000)

jdbc:postgresql://localhost:5432/tutorial_op?user=postgres&password=Taiga15!&ssl=false


Py4JJavaError: An error occurred while calling o65.showString.
: java.lang.IllegalStateException: SparkContext has been shutdown
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2385)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2414)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2433)
	at org.apache.spark.sql.execution.SparkPlan.executeTake(SparkPlan.scala:530)
	at org.apache.spark.sql.execution.SparkPlan.executeTake(SparkPlan.scala:483)
	at org.apache.spark.sql.execution.CollectLimitExec.executeCollect(limit.scala:61)
	at org.apache.spark.sql.Dataset.collectFromPlan(Dataset.scala:4334)
	at org.apache.spark.sql.Dataset.$anonfun$head$1(Dataset.scala:3316)
	at org.apache.spark.sql.Dataset.$anonfun$withAction$2(Dataset.scala:4324)
	at org.apache.spark.sql.execution.QueryExecution$.withInternalError(QueryExecution.scala:546)
	at org.apache.spark.sql.Dataset.$anonfun$withAction$1(Dataset.scala:4322)
	at org.apache.spark.sql.execution.SQLE

In [5]:
spark.stop()